In [1]:
import tkinter as tk
from tkinter import scrolledtext
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import skimage.io as io
import datetime
import numpy as np
from collections import Counter 

In [2]:
class GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.init_window()
        self.init_frame()
        self.config(menu=MenuBar(self))
        self.pointer = -1
        self.img_list = []
        
    def init_window(self):
        width = 1480
        height = 740
        screenwidth = self.winfo_screenwidth()  
        screenheight = self.winfo_screenheight()  
        size = '%dx%d+%d+%d'%(width, height, (screenwidth-width)/2, (screenheight-height)/8)
        self.geometry(size)
        self.title('Image Processing Toolkit')
        
    def init_frame(self):
        self.original = tk.LabelFrame(self, width=500, height=500, text='Original Image')  
        self.original.place(x=10, y=10)
        self.modified = tk.LabelFrame(self, width=500, height=500, text='Modified Image')  
        self.modified.place(x=520, y=10)
        self.original_hist = tk.LabelFrame(self, width=500, height=210, 
                                           text='Histogram of the Original Image')  
        self.original_hist.place(x=10, y=520)
        self.modified_hist = tk.LabelFrame(self, width=500, height=210, 
                                           text='Histogram of the Modified Image')  
        self.modified_hist.place(x=520, y=520)
        
        options = tk.LabelFrame(self, width=440, height=50, text='Options')
        options.place(x=1030, y=10)
        tk.Button(options, text="Load File", width=12, command=self.open_file,
                  relief=tk.GROOVE).pack(side=tk.LEFT, padx=3, pady=3)
        tk.Button(options, text="Save File", width=12, command=self.save_file,
                  relief=tk.GROOVE).pack(side=tk.LEFT, padx=3, pady=3)
        tk.Button(options, text="Undo", width=9, command=self.undo,
                  relief=tk.GROOVE).pack(side=tk.LEFT, padx=3, pady=3)
        tk.Button(options, text="Redo", width=9, command=self.redo,
                  relief=tk.GROOVE).pack(side=tk.LEFT, padx=3, pady=3)
        tk.Button(options, text="Reset", width=9, command=lambda: self.clear_image(1),
                  relief=tk.GROOVE).pack(side=tk.LEFT, padx=3, pady=3)
        
        messages = tk.LabelFrame(self, width=440, height=650, text='Messages')
        messages.place(x=1030, y=80)
        self.txt = scrolledtext.ScrolledText(messages, width=59, height=48, 
                                             bg='black', fg='white', wrap=tk.WORD,
                                             font=('Code New Roman', 10))
        self.txt.place(x=0, y=1)
        self.add_message('Welcome to the Image Processing Toolkit! You can either use the File menu or the button to load an image.')
        
    def add_message(self, string):
        self.txt.insert(tk.END,str(datetime.datetime.now())+'  '+string+'\n')   
    
    def open_file(self):
        if self.clear_image(level=2) == False:
            return
        else:
            pass
        path = tk.filedialog.askopenfilename()
        self.original_img = io.imread(path)
        self.display_image(original=True)
        self.add_message('Load image from '+path)
    
    def save_file(self):
        try:
            self.modified_fig
        except:
            tk.messagebox.showerror(title='Error', message='No modified image to save.', parent=self)
            return
        path = tk.filedialog.asksaveasfilename(defaultextension='.tif', 
                                               filetypes=[('TIFF', '*tif;*tiff'), 
                                                          ('JPEG', '*.jpg;*jpeg'), 
                                                          ('PNG', '*.png')])
        self.modified_fig.savefig(path)
        self.add_message('The modified image is saved.')
    
    def clear_image(self, level=0):
        if level >= 1 and self.pointer >= 0:
            msg = tk.messagebox.askquestion(title='Warning', icon = 'warning', parent=self,
                                            message='This operation cannot be undone.You will lose all unsaved changes. Are you sure?')
            if msg == 'no':
                return False
            else:
                pass
        try:
            if level == 2:
                self.original_canvas.get_tk_widget().pack_forget()
                self.original_hist_canvas.get_tk_widget().pack_forget()
                del self.original_img
            if level >= 1:
                self.add_message('Reset to the original image.')
                self.pointer = -1
                self.img_list = []
            self.modified_canvas.get_tk_widget().pack_forget()
            self.modified_hist_canvas.get_tk_widget().pack_forget()
            del self.modified_fig
        except AttributeError: 
            pass
        
    def display_image(self, original=False):
        ticks = [i-1 if i==256 else i for i in range(0,257,32)]
        if original == True:
            fig = Figure(figsize=(5, 4.8), dpi=100, constrained_layout=True)
            fig.add_subplot(111, frame_on=False, xticks=[], yticks=[]).\
                imshow(self.original_img, cmap='gray', vmin=0, vmax=255)
            self.original_canvas = FigureCanvasTkAgg(fig, master=self.original)   
            self.original_canvas.draw() 
            self.original_canvas.get_tk_widget().pack()
            
            hist = Figure(figsize=(5, 2), dpi=100, constrained_layout=True)
            hist.add_subplot(111, xticks=ticks).\
                hist(np.ravel(self.original_img), bins=256)
            self.original_hist_canvas = FigureCanvasTkAgg(hist, master=self.original_hist)   
            self.original_hist_canvas.draw() 
            self.original_hist_canvas.get_tk_widget().pack()
        else:
            img = self.img_list[self.pointer].astype('uint8')
            self.modified_fig = Figure(figsize=(5, 4.8), dpi=100, constrained_layout=True)
            self.modified_fig.add_subplot(111, frame_on=False, xticks=[], yticks=[]).\
                imshow(img, cmap='gray', vmin=0, vmax=255)
            self.modified_canvas = FigureCanvasTkAgg(self.modified_fig, master=self.modified)   
            self.modified_canvas.draw() 
            self.modified_canvas.get_tk_widget().pack()
        
            hist = Figure(figsize=(5, 2), dpi=100, constrained_layout=True)
            hist.add_subplot(111, xticks=ticks).\
                hist(np.ravel(img), bins=256)
            self.modified_hist_canvas = FigureCanvasTkAgg(hist, master=self.modified_hist)   
            self.modified_hist_canvas.draw() 
            self.modified_hist_canvas.get_tk_widget().pack()
    
    def modify_image(self, method, *args):
        if self.pointer >= 0: 
            img = self.img_list[self.pointer]
        else: 
            try:
                img = self.original_img
            except AttributeError:
                tk.messagebox.showerror(title='Error', message='Please load an image first.', parent=self)
                return
        try:
            modified_img = method(img)
        except:
            self.add_message('Fail to perform this operation.')
            return
        self.pointer += 1
        if self.pointer >= 0:
            self.img_list = self.img_list[:self.pointer]
            self.clear_image() 
        else:
            pass
        self.img_list.append(modified_img)
        self.display_image()
        self.add_message('Successfully executed.')
    
    def undo(self):
        if self.pointer >= 0:
            self.add_message('Undo.')
            self.pointer -= 1
            self.clear_image()
            if self.pointer >= 0:
                self.display_image()
            else:
                pass
        else:
            tk.messagebox.showerror(title='Error', message='No previous operation.', parent=self)
            
    def redo(self):
        if self.pointer+1 < len(self.img_list):
            self.add_message('Redo.')
            self.pointer += 1
            self.clear_image()
            self.display_image()
        else:
            tk.messagebox.showerror(title='Error', message='No following operation.', parent=self)
    
    # Image processing functions
    def nearest_neighbor(self, f, x, y):
        a = int(x)
        b = int(y)
        c = x-a
        d = y-b
        return f[a+int(c*2), b+int(d*2)]
    
    def bilinear(self, f, x, y):
        a = int(x)
        b = int(y)
        c = x-a
        d = y-b
        
        M, N = f.shape
        f_pad = np.zeros((M+1, N+1))
        f_pad[:M, :N] = f
        A = f_pad[a:a+2, b:b+2]
        X = np.array([1-c, c])
        Y = np.array([1-d, d])
        return X@A@Y
    
    def bicubic(self, f, x, y):
        a = int(x)
        b = int(y)
        c = x-a
        d = y-b
        
        M, N = f.shape
        f_pad = np.zeros((M+3, N+3))
        f_pad[1:M+1, 1:N+1] = f
        w = lambda x: 1-2*(x**2)+(x**3) if x<=1 else 4-8*x+5*(x**2)-(x**3)
        A = f_pad[a:a+4, b:b+4]
        X = np.array([w(c+1), w(c), w(1-c), w(2-c)])
        Y = np.array([w(d+1), w(d), w(1-d), w(2-d)])
        return X@A@Y
    
    def scaling(self, f):
        d = Dialog_scaling(self, 'Scaling')
        M, N = f.shape
        m = int((M-1)*d.k)+1
        n = int((N-1)*d.k)+1
        
        g = np.zeros((m, n))
        for i in range(m):
            for j in range(n):
                x = i/d.k
                y = j/d.k
                if d.interpolation == 0:
                    g[i, j] = self.nearest_neighbor(f, x, y)
                elif d.interpolation == 1:
                    g[i, j] = self.bilinear(f, x, y)
                else:
                    g[i, j] = self.bicubic(f, x, y)
        return g

    def rotation(self, f):
        d = Dialog_rotation(self, 'Rotation')
        M, N = f.shape
        F_x = lambda x, y, theta: x*np.cos(theta)-y*np.sin(theta)
        F_y = lambda x, y, theta: x*np.sin(theta)+y*np.cos(theta)
        F_inv_x = lambda x, y, theta: x*np.cos(theta)+y*np.sin(theta)
        F_inv_y = lambda x, y, theta: -x*np.sin(theta)+y*np.cos(theta)
        m_max = max(0, F_x(0,N-1,d.theta), F_x(M-1,0,d.theta), F_x(M-1,N-1,d.theta))
        m_min = min(0, F_x(0,N-1,d.theta), F_x(M-1,0,d.theta), F_x(M-1,N-1,d.theta))
        n_max = max(0, F_y(0,N-1,d.theta), F_y(M-1,0,d.theta), F_y(M-1,N-1,d.theta))
        n_min = min(0, F_y(0,N-1,d.theta), F_y(M-1,0,d.theta), F_y(M-1,N-1,d.theta))
        m = int(m_max-m_min)+1
        n = int(n_max-n_min)+1
        
        g = np.zeros((m, n))
        for i in range(m):
            for j in range(n):
                x = F_inv_x(i+m_min, j+n_min, d.theta)
                y = F_inv_y(i+m_min, j+n_min, d.theta)
                if d.interpolation == 0:
                    if 0<=x<=M-1 and 0<=y<=N-1:
                        g[i, j] = self.nearest_neighbor(f, x, y)
                    else:
                        pass
                elif d.interpolation == 1:
                    if 0<=x<=M-1 and 0<=y<=N-1:
                        g[i, j] = self.bilinear(f, x, y)
                    else:
                        pass
                else:
                    if 0<=x<=M-1 and 0<=y<=N-1:
                        g[i, j] = self.bicubic(f, x, y)
                    else:
                        pass      
        return g
    
    def negative_transformation(self, f):
        self.add_message('Processing negative transformation...')
        return 255-f
    
    def contrast_stretching(self, f):
        self.add_message('Processing contrast stretching...')
        g = (f-f.min())/(f.max()-f.min())*255
        return g
    
    def intensity_level_slicing(self, f):
        d = Dialog_intensity_level_slicing(self, 'Intensity Level Slicing')
        if d.bool == 0:
            g = np.where((f>d.a)&(f<d.b), d.s, f)
        else:
            g = np.where((f>d.a)&(f<d.b), d.s, d.t)
        return g
        
    def power_law(self, f):
        d = Dialog_power_law(self, 'Power-Law (Gamma) Transformation')
        g = (f/255)**d.a*255
        return g
        
    def histogram_equalization(self, f):
        self.add_message('Processing histogram equalization...')
        c = Counter(np.ravel(f).astype('uint8'))
        g = np.zeros(f.shape)
        s = 0
        n = f.size
        for i in range(256):
            s += 255*c[i]/n
            g[f==i] = round(s)
        return g
        
    def convolution(self, f, w, padding):
        M, N = f.shape
        a = int(w.shape[0]/2)
        b = int(w.shape[1]/2)
        m = 2*a+1
        n = 2*b+1
        v = np.zeros((m,n))
        v[:w.shape[0], :w.shape[1]] = w
        g = np.zeros([M+2*a, N+2*b])
        g[a:M+a, b:N+b] = f
        if padding == 1:
            g[0:a, b:N+b] = g[2*a:a:-1, b:N+b]
            g[M+a:, b:N+b] = g[M+a-2:M-2:-1, b:N+b]
            g[:, 0:b] = g[:, 2*b:b:-1]
            g[:, N+b:] = g[:, N+b-2:N-2:-1]
        elif padding == 2:
            g[0:a, b:N+b] = g[a, b:N+b]
            g[M+a:, b:N+b] = g[M+a-1, b:N+b]
            g[:, 0:b] = np.tile(g[:, b], b).reshape(b, M+2*a).T
            g[:, N+b:] = np.tile(g[:, N+b-1], b).reshape(b, M+2*a).T
        else:
            pass
        h = np.zeros((M,N))
        for i in range(m):
            for j in range(n):
                h += v[m-i-1,n-j-1]*g[i:i+M, j:j+N]
        return h
    
    def box_filtering_seperate(self, f, size, padding):
        w_1 = np.ones([size,1])/size
        w_2 = np.ones([1,size])/size
        return self.convolution(self.convolution(f, w_1, padding), w_2, padding)

    def box_filtering(self, f):
        d = Dialog_box_filtering(self, 'Box Filtering')
        return self.box_filtering_seperate(f, d.size, d.pad)
        
    def gaussian_filtering_seperate(self, f, size, padding):
        sigma = np.floor(size/6)
        w = np.array([np.exp(-i**2/sigma**2/2) for i in range(-int(size/2), int((size+1)/2))])
        w_sum = np.sum(w)
        w_1 = w.reshape(size,1)/w_sum
        w_2 = w.reshape(1,size)/w_sum
        return self.convolution(self.convolution(f, w_1, padding), w_2, padding)
        
    def gaussian_filtering(self, f):
        d = Dialog_gaussian_filtering(self, 'Gaussian Filtering')
        return self.gaussian_filtering_seperate(f, d.size, d.pad)
        
    def median_filtering(self, f):
        d = Dialog_median_filtering(self, 'Median Filtering')
        M,N = f.shape
        a = int(d.size/2)
        b = int((d.size-1)/2)
        g = np.empty((M+2*a,N+2*b))
        g.fill(np.nan)
        g[a:M+a, b:N+b] = f
        h = np.zeros((M, N))
        for i in range(M):
            for j in range(N):
                h[i, j] = np.nanmedian(g[i:i+2*a+1, j:j+2*b+1])
        return h
        
    def laplacian(self, f):
        d = Dialog_laplacian(self, 'Laplacian Filtering')
        if d.bool == 1:
            w = np.array([[1, 1, 1],
                          [1, -8, 1],
                          [1, 1, 1]])
        else:
            w = np.array([[0, 1, 0],
                          [1, -4, 1],
                          [0, 1, 0]])
        M,N = f.shape
        g = np.zeros((M-2, N-2))
        for i in range(3):
            for j in range(3):
                g += w[i,j]*f[i:i+M-2, j:j+N-2]
        h = np.zeros(f.shape)
        h[1:M-1, 1:N-1] = g
        return np.clip(f-d.k*h, 0, 255)
    
    def highboost_filtering(self, f):
        d = Dialog_highboost_filtering(self, 'Unsharp Masking and Highboost Filtering')
        if d.bool == 0:
            g = self.box_filtering_seperate(f, d.size, d.pad)
        else:
            g = self.gaussian_filtering_seperate(f, d.size, d.pad)
        mask = f-g
        return np.clip(f+d.k*mask, 0, 255)

In [3]:
class MenuBar(tk.Menu):
    def __init__(self, parent):
        tk.Menu.__init__(self, parent)
        
        file = tk.Menu(self, tearoff=False)
        file.add_command(label='Load Image From File', command=parent.open_file)
        file.add_command(label='Save Modified Image As', command=parent.save_file)
        file.add_separator()
        file.add_command(label='Undo', command=parent.undo)
        file.add_command(label='Redo', command=parent.redo)
        file.add_command(label='Reset to the original image', command=lambda: parent.clear_image(1))
        file.add_separator()
        file.add_command(label='Scaling', command=lambda: parent.modify_image(parent.scaling))
        file.add_command(label='Rotation', command=lambda: parent.modify_image(parent.rotation))
        file.add_separator()
        file.add_command(label='Exit', command=parent.destroy)
        self.add_cascade(label='File', menu=file)
        
        point = tk.Menu(self, tearoff=False)
        point.add_command(label='Negative Transformation',
                          command=lambda: parent.modify_image(parent.negative_transformation))
        point.add_command(label='Intensity Level Slicing',
                          command=lambda: parent.modify_image(parent.intensity_level_slicing))
        point.add_command(label='Contrast Stretching', 
                          command=lambda: parent.modify_image(parent.contrast_stretching))
        point.add_command(label='Power-Law Transformation', 
                          command=lambda: parent.modify_image(parent.power_law))
        point.add_command(label='Histogram Equalization', 
                          command=lambda: parent.modify_image(parent.histogram_equalization))
        self.add_cascade(label='Point Processing', menu=point)
        
        neighborhood = tk.Menu(self, tearoff=False)
        neighborhood.add_command(label='Box Filtering',
                                 command=lambda: parent.modify_image(parent.box_filtering))
        neighborhood.add_command(label='Gaussian Filtering',
                                 command=lambda: parent.modify_image(parent.gaussian_filtering))
        neighborhood.add_command(label='Median filtering',
                                 command=lambda: parent.modify_image(parent.median_filtering))
        neighborhood.add_separator()
        neighborhood.add_command(label='Laplacian Filtering',
                                 command=lambda: parent.modify_image(parent.laplacian))
        neighborhood.add_command(label='Highboost Filtering', 
                                 command=lambda: parent.modify_image(parent.highboost_filtering))
        self.add_cascade(label='Neighborhood Processing', menu=neighborhood)

In [4]:
class Dialog(tk.Toplevel):
    def __init__(self, parent, name=''):
        tk.Toplevel.__init__(self, parent)
        self.transient(parent)
        self.parent = parent
        self.name = name
        self.title(name)
        self.geometry('+600+300')
        self.init_widgets()
        self.init_buttons()
        self.grab_set()
        self.protocol("WM_DELETE_WINDOW", self.cancel_click)
        self.wait_window(self)
    
    def init_buttons(self):
        buttons = tk.Frame(self)
        tk.Button(buttons, text="Apply", width=10, command=self.apply_click
                  ).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(buttons, text="Cancel", width=10, command=self.cancel_click
                  ).pack(side=tk.LEFT, padx=5, pady=5)
        buttons.grid(row=4, column=0, padx=10, pady=5)
    
    def apply_click(self):
        try:
            if self.validate() is not True:
                tk.messagebox.showerror(title='Error', message='The input is invalid.', parent=self)
                return
        except ValueError:
            tk.messagebox.showerror(title='Error', message='The input is invalid.', parent=self)
            return
        self.withdraw()
        self.parent.add_message('Processing ' + self.name + '...')
        self.update_idletasks()
        self.parent.focus_set()
        self.destroy()
        
    def cancel_click(self):
        self.parent.focus_set()
        self.destroy()

In [5]:
class Dialog_scaling(Dialog): 
    def init_widgets(self):
        frame1 = tk.Frame(self)
        tk.Label(frame1, text='Enter the scale factor (must be positive): ').\
            grid(row=0, column=0, sticky='w')
        self.k_entry = tk.Entry(frame1, width=6)
        self.k_entry.grid(row=0, column=1)
        self.k_entry.focus_set()
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Interpolation Options', width=500) 
        self.interpolation_entry = tk.IntVar()
        tk.Radiobutton(frame2, text='Nearest Neighbor',variable=self.interpolation_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame2, text='Bilinear',variable=self.interpolation_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        tk.Radiobutton(frame2, text='Bicubic',variable=self.interpolation_entry, 
                       value=2, anchor='w').grid(row=0, column=2, sticky='w')
        frame2.grid(row=1, column=0, padx=10, pady=5, sticky='w')
    
    def validate(self):
        self.interpolation = self.interpolation_entry.get()
        self.k = float(self.k_entry.get())
        if self.k > 0:
            return True

In [6]:
class Dialog_rotation(Dialog): 
    def init_widgets(self):
        frame1 = tk.Frame(self)
        tk.Label(frame1, text='Enter the counterclockwise angle: ').\
            grid(row=0, column=0, sticky='w')
        self.theta_entry = tk.Entry(frame1, width=6)
        self.theta_entry.grid(row=0, column=1)
        self.theta_entry.focus_set()
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Interpolation Options', width=500) 
        self.interpolation_entry = tk.IntVar()
        tk.Radiobutton(frame2, text='Nearest Neighbor',variable=self.interpolation_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame2, text='Bilinear',variable=self.interpolation_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        tk.Radiobutton(frame2, text='Bicubic',variable=self.interpolation_entry, 
                       value=2, anchor='w').grid(row=0, column=2, sticky='w')
        frame2.grid(row=1, column=0, padx=10, pady=5, sticky='w')
        
    def validate(self):
        self.interpolation = self.interpolation_entry.get()
        self.theta = np.radians(float(self.theta_entry.get()))
        return True

In [7]:
class Dialog_intensity_level_slicing(Dialog):   
    def init_widgets(self):
        frame1 = tk.LabelFrame(self, text='Enter the range of intensities to be sliced (between 0 and 255)')
        self.a_entry = tk.Entry(frame1, width=6)
        self.a_entry.grid(row=0, column=1, padx=5, pady=5)
        self.a_entry.focus_set()
        tk.Label(frame1, text='~', width=3).grid(row=0, column=2, pady=5)
        self.b_entry = tk.Entry(frame1, width=6)
        self.b_entry.grid(row=0, column=3, padx=5, pady=5)
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Enter the intensities for the modified image (between 0 and 255)')
        tk.Label(frame2, text='Change the sliced intensities to: ').\
            grid(row=0, column=0, padx=3, pady=5, sticky='w')
        self.s_entry = tk.Entry(frame2, width=6)
        self.s_entry.grid(row=0, column=1, sticky='w')
        tk.Label(frame2, text='-----------------------------------------------').\
            grid(row=1, column=0, padx=3, sticky='w')
        self.bool_entry = tk.IntVar()
        tk.Radiobutton(frame2, text='Leave the other intensities unchanged', variable=self.bool_entry, 
                       value=0, anchor='w').grid(row=2, column=0, pady=2, sticky='w')
        tk.Radiobutton(frame2, text='Change the other intensities to :', variable=self.bool_entry, 
                       value=1, anchor='w').grid(row=3, column=0, pady=2, sticky='w')
        self.t_entry = tk.Entry(frame2, width=6, textvariable=tk.DoubleVar(value=0))
        self.t_entry.grid(row=3, column=1, pady=5, sticky='w')
        frame2.grid(row=1, column=0, padx=10, pady=2, sticky='w')
    
    def validate(self):
        self.a = float(self.a_entry.get())
        self.b = float(self.b_entry.get())
        self.s = float(self.s_entry.get())
        self.t = float(self.t_entry.get())
        self.bool = self.bool_entry.get()
        if self.a < self.b and 0<=self.a<=255 and 0<=self.b<=255 and 0<=self.s<=255 and 0<=self.t*self.bool<=255:
            return True

In [8]:
class Dialog_power_law(Dialog):
    def init_widgets(self):
        frame = tk.Frame(self)
        tk.Label(frame, text='Enter the value of gamma (must be positive):').\
            grid(row=0, column=0, sticky='w')
        self.a_entry = tk.Entry(frame, width=6, relief=tk.GROOVE)
        self.a_entry.grid(row=0, column=1)
        self.a_entry.focus_set()
        frame.grid(row=0, column=0, padx=10, pady=5)
    
    def validate(self):
        self.a = float(self.a_entry.get())
        if self.a > 0:
            return True

In [9]:
class Dialog_box_filtering(Dialog):  
    def init_widgets(self):
        frame1 = tk.LabelFrame(self, text='Kernel Size') 
        self.size_entry = tk.IntVar()
        tk.Scale(frame1, orient=tk.HORIZONTAL, length=500, from_=3, to=21,
                 tickinterval=2, variable=self.size_entry).pack()
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Padding Options') 
        self.pad_entry = tk.IntVar()
        tk.Radiobutton(frame2, text='Zero Padding',variable=self.pad_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame2, text='Mirror Padding',variable=self.pad_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        tk.Radiobutton(frame2, text='Replicate Padding',variable=self.pad_entry, 
                       value=2, anchor='w').grid(row=0, column=2, sticky='w')
        frame2.grid(row=1, column=0, padx=10, pady=5, sticky='w')
    
    def validate(self):
        self.size = self.size_entry.get()
        self.pad = self.pad_entry.get()
        return True

In [10]:
class Dialog_gaussian_filtering(Dialog):  
    def init_widgets(self):
        frame1 = tk.LabelFrame(self, text='Kernel Size') 
        self.size_entry = tk.IntVar()
        tk.Scale(frame1, orient=tk.HORIZONTAL, length=500, from_=7, to=97,
                 tickinterval=10, variable=self.size_entry).pack()
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Padding Options') 
        self.pad_entry = tk.IntVar()
        tk.Radiobutton(frame2, text='Zero Padding',variable=self.pad_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame2, text='Mirror Padding',variable=self.pad_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        tk.Radiobutton(frame2, text='Replicate Padding',variable=self.pad_entry, 
                       value=2, anchor='w').grid(row=0, column=2, sticky='w')
        frame2.grid(row=1, column=0, padx=10, pady=5, sticky='w')
        
    def validate(self):
        self.size = self.size_entry.get()
        self.pad = self.pad_entry.get()
        return True

In [11]:
class Dialog_median_filtering(Dialog):    
    def init_widgets(self):
        frame = tk.LabelFrame(self, text='Kernel Size') 
        self.size_entry = tk.IntVar()
        tk.Scale(frame, orient=tk.HORIZONTAL, length=500, from_=3, to=21,
                            tickinterval=2, variable=self.size_entry).pack()
        frame.grid(row=0, column=0, padx=10, pady=5)
    
    def validate(self):
        self.size = self.size_entry.get()
        return True

In [12]:
class Dialog_laplacian(Dialog):  
    def init_widgets(self):
        frame = tk.Frame(self)
        tk.Label(frame, text='Enter the weighted portion of the Laplacian image back to the original image (must be positive): ').\
            grid(row=0, column=0, sticky='w')
        self.k_entry = tk.Entry(frame, width=6, textvariable=tk.DoubleVar(value=1))
        self.k_entry.grid(row=0, column=1)
        self.k_entry.focus_set()
        self.bool_entry = tk.IntVar()
        tk.Checkbutton(frame,text='Sharpen in the diagonal directions',
                    variable = self.bool_entry, onvalue=1, offvalue=0).grid(row=1, column=0)
        frame.grid(row=0, column=0, padx=10, pady=5)
    
    def validate(self): 
        self.k = float(self.k_entry.get())
        self.bool = self.bool_entry.get()
        if self.k > 0:
            return True

In [13]:
class Dialog_highboost_filtering(Dialog): 
    def init_widgets(self):
        frame1 = tk.LabelFrame(self, text='Kernel Type for the blurred image', width=500) 
        self.bool_entry = tk.IntVar()
        tk.Radiobutton(frame1, text='Box Kernel', variable=self.bool_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame1, text='Gaussian Kernel', variable=self.bool_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        frame1.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        
        frame2 = tk.LabelFrame(self, text='Kernel Size for the blurred image', width=500) 
        self.size_entry = tk.IntVar()
        tk.Scale(frame2, orient=tk.HORIZONTAL, length=500, from_=7, to=97,
                            tickinterval=10, variable=self.size_entry).pack()
        frame2.grid(row=1, column=0, padx=10, pady=5, sticky='w')
        
        frame3 = tk.LabelFrame(self, text='Padding Options for the blurred image', width=500) 
        self.pad_entry = tk.IntVar()
        tk.Radiobutton(frame3, text='Zero Padding',variable=self.pad_entry, 
                       value=0, anchor='w').grid(row=0, column=0, sticky='w')
        tk.Radiobutton(frame3, text='Mirror Padding',variable=self.pad_entry, 
                       value=1, anchor='w').grid(row=0, column=1, sticky='w')
        tk.Radiobutton(frame3, text='Replicate Padding',variable=self.pad_entry, 
                       value=2, anchor='w').grid(row=0, column=2, sticky='w')
        frame3.grid(row=2, column=0, padx=10, pady=5, sticky='w')
        
        frame4 = tk.Frame(self, width=500) 
        tk.Label(frame4, text='Enter the weighted portion of the mask back to the original image (must be positive): ').\
            grid(row=0, column=0, sticky='w')
        self.k_entry = tk.Entry(frame4, width=6, textvariable=tk.DoubleVar(value=1))
        self.k_entry.grid(row=0, column=1)
        frame4.grid(row=3, column=0, padx=10, pady=5, sticky='w')
    
    def validate(self):
        self.bool = self.bool_entry.get()
        self.size = self.size_entry.get()
        self.pad = self.pad_entry.get()
        self.k = float(self.k_entry.get())
        if self.k > 0:
            return True

In [14]:
if __name__ == "__main__":
    gui = GUI()
    gui.mainloop()